In [1]:
from sklearn import datasets
bostonDF, y = datasets.fetch_openml('boston', return_X_y=True)

/Users/song-yeongmin/anaconda3/lib/python3.11/site-packages/sklearn/datasets/_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
/Users/song-yeongmin/anaconda3/lib/python3.11/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
bostonDF['PRICE'] = y

In [4]:
print(bostonDF.shape)
bostonDF.head()

(506, 14)
